## Homework: Multilingual Embedding-based Machine Translation (7 points)

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully). 

For our system we choose two kindred Slavic languages: Ukrainian and Russian. 

### Feel the difference!

(_синій кіт_ vs. _синій кит_)

![blue_cat_blue_whale.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/blue_cat_blue_whale.png)

### Fragment of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [2]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

/Users/d.kulemin/Projects/nlp-for-you/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Download embeddings here:
* [cc.uk.300.vec.zip](https://yadi.sk/d/9CAeNsJiInoyUA)
* [cc.ru.300.vec.zip](https://yadi.sk/d/3yG0-M4M8fypeQ)

Load embeddings for ukrainian and russian.

In [4]:
uk_emb = KeyedVectors.load_word2vec_format("data/cc.uk.300.vec")

In [5]:
ru_emb = KeyedVectors.load_word2vec_format("data/cc.ru.300.vec")

In [6]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240028262138367),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401415944099426)]

In [7]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999998807907104),
 ('липень', 0.9096440076828003),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992519974708557),
 ('жовтень', 0.8810409307479858),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592805862426758),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

In [8]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293080925941467),
 ('офор', 0.23256802558898926),
 ('deteydlya', 0.230317160487175),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608886659145355),
 ('подход', 0.2230587750673294),
 ('иболее', 0.22003725171089172),
 ('2015Александр', 0.21872763335704803),
 ('конструктивен', 0.21796567738056183)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [132]:
def load_word_pairs(filename):
    uk_ru_pairs, uk_vectors, ru_vectors = [
        (uk_ru_pairs, uk_vectors, ru_vectors)
        # for (uk_ru_pairs, uk_vectors, ru_vectors) in [([], np.array([]), np.array([]))]
        for (uk_ru_pairs, uk_vectors, ru_vectors) in [([], [], [])]
        for line in open(filename, "r")
        for (uk, ru) in [line.rstrip().split("\t")]
        if uk in uk_emb and ru in ru_emb
        for _ in [(
            uk_ru_pairs.append((uk, ru)),
            # np.hstack((uk_vectors, uk_emb[uk])),
            # np.hstack((ru_vectors, ru_emb[ru])),
            uk_vectors.append(uk_emb[uk]),
            ru_vectors.append(ru_emb[ru])
        )]
    ][0]
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [59]:
# def load_word_pairs(filename):
#     uk_ru_pairs = []
#     uk_vectors = []
#     ru_vectors = []
#     with open(filename, "r") as inpf:
#         for line in inpf:
#             uk, ru = line.rstrip().split("\t")
#             if uk not in uk_emb or ru not in ru_emb:
#                 continue
#             uk_ru_pairs.append((uk, ru))
#             uk_vectors.append(uk_emb[uk])
#             ru_vectors.append(ru_emb[ru])
#     return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [134]:
uk_ru_train, X_train, Y_train = load_word_pairs("data/ukr_rus.train.txt")

In [61]:
uk_ru_test, X_test, Y_test = load_word_pairs("data/ukr_rus.test.txt")

## Embedding space mapping

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

In Greek mythology, Procrustes or "the stretcher" was a rogue smith and bandit from Attica who attacked people by stretching them or cutting off their legs, so as to force them to fit the size of an iron bed. We make same bad things with source embedding space. Our Procrustean bed is target embedding space.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

![procrustes.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/procrustes.png)

But wait...$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [137]:
from sklearn.linear_model import LinearRegression

procrustes_bed = LinearRegression(fit_intercept=False).fit(X_train, Y_train)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [138]:
august = procrustes_bed.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541286587715149),
 ('июнь', 0.8411202430725098),
 ('март', 0.839699387550354),
 ('сентябрь', 0.8359869718551636),
 ('февраль', 0.8329297304153442),
 ('октябрь', 0.8311846852302551),
 ('ноябрь', 0.8278924226760864),
 ('июль', 0.823452889919281),
 ('август', 0.8120501637458801),
 ('декабрь', 0.803900420665741)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [149]:
num_matches = 0
num_matches += 'апрель' in [w for w, sc in ru_emb.most_similar(august, topn=1)]

In [156]:
X_train[1]

array([ 1.697e-01, -3.190e-02,  2.440e-02,  4.200e-02, -4.530e-02,
       -3.620e-02, -1.890e-02, -5.400e-03, -1.082e-01,  2.860e-02,
       -6.860e-02, -8.700e-03,  1.023e-01,  1.080e-02, -4.100e-02,
       -1.390e-02,  1.146e-01, -3.020e-02, -9.200e-03,  9.610e-02,
        1.047e-01, -1.900e-02,  5.730e-02,  4.490e-02, -6.130e-02,
       -5.000e-03,  1.100e-01,  1.920e-02, -1.019e-01,  3.580e-02,
       -1.020e-02,  6.730e-02,  3.010e-02, -4.330e-02, -5.800e-03,
        3.390e-02, -9.900e-02, -4.170e-02, -2.260e-02,  2.920e-02,
       -4.200e-02, -4.870e-02, -3.440e-02,  5.260e-02, -1.400e-03,
       -8.020e-02,  5.530e-02, -3.370e-02,  2.440e-02,  1.070e-02,
        4.200e-03, -6.960e-02,  1.100e-02,  2.920e-02, -3.190e-02,
       -2.360e-02,  4.600e-02, -6.080e-02,  4.270e-02, -3.480e-02,
        1.540e-02, -2.940e-02, -2.510e-02,  3.850e-02, -5.550e-02,
       -2.680e-02, -2.900e-03,  2.260e-02, -9.650e-02, -6.380e-02,
        8.250e-02,  3.700e-03,  7.060e-02, -6.560e-02, -1.590e

In [150]:
num_matches

1

In [163]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, ((_, ru), emb) in enumerate(zip(pairs, mapped_vectors)):
        num_matches += ru in [w for w, _ in ru_emb.most_similar(emb, topn=topn)]
    precision_val = num_matches / len(pairs)
    return precision_val


In [164]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [165]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [167]:
precision_top1 = precision(uk_ru_test, procrustes_bed.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, procrustes_bed.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

## Making it better (orthogonal Procrustean problem)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [173]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    U, _, V = np.linalg.svd(X_train.T @ Y_train)
    return U @ V
    

In [174]:
W = learn_transform(X_train, Y_train)

In [175]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8237907886505127),
 ('сентябрь', 0.8049710988998413),
 ('март', 0.8025652170181274),
 ('июнь', 0.8021840453147888),
 ('октябрь', 0.8001735210418701),
 ('ноябрь', 0.793448269367218),
 ('февраль', 0.7914120554924011),
 ('июль', 0.7908107042312622),
 ('август', 0.7891015410423279),
 ('декабрь', 0.7686371803283691)]

In [176]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

## UK-RU Translator

Now we are ready to make simple word-based translator: for each word in source language in shared embedding space we find the nearest in target language.


In [177]:
with open("data/fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [236]:
from string import punctuation

In [256]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    return " ".join([
        ru_emb.most_similar(uk_emb[word] @ W, topn=1)[0][0] if word in uk_emb else '<unk>'
        for word in "".join([
            f' {s}' if s in punctuation else s
            for s in sentence
        ]).split()
    ])

In [254]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [257]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк – <unk>

src: як була собі лисичка та зробила хатку, та й живе. а це приходять холоди. от лисичка замерзла та й побігла в село вогню добувать, щоб витопити. прибігає до одної баби та й каже:
dst: как была себе лисичка и сделала избушку , и и живет . а оно приходят морозы . из лисичка замерзла и и побежала во село огня <unk> , чтобы <unk> . прибегает к одной бабы и и говорит :

src: — здорові були, бабусю! з неділею... позичте мені огню, я вам одслужу.
dst: — здоровые были , бабушку ! со воскресеньем . . . <unk> мне огня , мной тебе <unk> .

src: — добре, — каже, — лисичко - сестричко. сідай погрійся трохи, поки я пиріжечки повибираю з печі!
dst: — хорошо , — говорит , — <unk> – сестричка . садись <unk> немного , пока мной пирожки <unk> со печи !

src: а баба макові пиріжки пекла. от баба вибирає пиріжки та на столі кладе, щоб прохололи; а лисичка підгляділа та за пиріг, та з хати... виїла мачок із середини, а ту

Not so bad, right? We can easily improve translation using language model and not one but several nearest neighbours in shared embedding space. But next time.

## Would you like to learn more?

### Articles:
* [Exploiting Similarities among Languages for Machine Translation](https://arxiv.org/pdf/1309.4168)  - entry point for multilingual embedding studies by Tomas Mikolov (the author of W2V)
* [Offline bilingual word vectors, orthogonal transformations and the inverted softmax](https://arxiv.org/pdf/1702.03859) - orthogonal transform for unsupervised MT
* [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087)
* [Loss in Translation: Learning Bilingual Word Mapping with a Retrieval Criterion](https://arxiv.org/pdf/1804.07745)
* [Unsupervised Alignment of Embeddings with Wasserstein Procrustes](https://arxiv.org/pdf/1805.11222)

### Repos (with ready-to-use multilingual embeddings):
* https://github.com/facebookresearch/MUSE

* https://github.com/Babylonpartners/fastText_multilingual -